<a href="https://colab.research.google.com/github/Jiali03713/LLM-with-RAG/blob/main/LLM_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Setup

In [1]:
!git clone https://github.com/Jiali03713/LLM-with-RAG
!pip install --upgrade pip
!pip install --upgrade transformers sentencepiece datasets[audio]
!pip install -r /content/LLM-with-RAG/requirements.txt
!pip install playsound


Cloning into 'LLM-with-RAG'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 247 (delta 32), reused 38 (delta 12), pack-reused 185 (from 1)
Receiving objects: 100% (247/247), 8.21 MiB | 4.43 MiB/s, done.
Resolving deltas: 100% (127/127), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 113.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenize

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=9439679f2f76b2f638061129851c0d485caad52eeba034e2fb3c2238c7325bb5
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound
ERROR: Operation cancelled by user


In [1]:
from pymilvus import Collection, connections, utility
from pymilvus import MilvusClient

milvus_ip = '34.69.5.20'
from pymilvus import MilvusClient, DataType
# Replace with your Google Cloud VM's public IP
client = MilvusClient(
    uri="http://34.69.5.20:19530",
    token="root:Milvus"
)


# /content/LLM-with-RAG/final.pdf

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: d4fc1c15650a44dfb7674c2ff46b57f3


###text_to_speech


In [2]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset
from IPython.display import HTML, display, Audio

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [3]:
def text_to_speech(text):
    text = ''.join(text)

    inputs = processor(text=text, return_tensors="pt", padding=True, truncation=True)

    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    sf.write("speech.wav", speech.numpy(), samplerate=16000)
    display(Audio("speech.wav", autoplay=True))

###pdf_to_txt



In [4]:
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from contextlib import contextmanager
import pdfplumber

def save_text_to_file(texts, file_name):
    if os.path.exists(file_name + ".txt"):
        os.remove(file_name + ".txt")

    with open(file_name + ".txt", 'a', encoding='utf-8') as file:
        for text in texts:
            file.write(text + "\n")

    return file_name + ".txt"


# Extract text with pdfPlumber
def extract_text_pdfplumber(pdf_path):
    output_folder = "/content/pdfReader_output"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ''
            count = 0
            for page in pdf.pages:
                count += 1
                page_content = page.extract_text().replace("\n", " ")
                text += page_content + " "

    except FileNotFoundError as e:
        print(f"File not found: {e}")
        return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    file_name = os.path.splitext(os.path.basename(pdf_path))[0]
    text_list = text.split(".")
    text_list = [element for element in text_list if element and isinstance(element, str) and len(element) > 10]

    save_text_to_file(text_list, os.path.join(output_folder, file_name))

    print(f"Text successfully extracted and saved to {os.path.join(output_folder, file_name)}.txt")
    output_path = os.path.join(output_folder, file_name) + ".txt"

    return file_name, output_path


@contextmanager
def use_tkinter():
    root = Tk()
    root.withdraw()
    try:
        yield root
    finally:
        root.quit()

def select_pdf():
    with use_tkinter():
        pdf_file = askopenfilename(title="Select PDF file", filetypes=[("PDF files", "*.pdf")])

        if pdf_file:
            output_path = extract_text_pdfplumber(pdf_file)
            return output_path
        else:
            print("No file selected or invalid file.")


def tts_run(file_path):
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    output_folder = "/content/pdfReader_output"

    output_path = f"{os.path.join(output_folder, file_name)}.txt"
    print(output_path)

    if os.path.exists(output_path):
        print(output_path)
        return file_name, output_path
    else:
        file_path = input("Enter the path of your pdf file: ")
        filename, output_path = extract_text_pdfplumber(file_path)
        return filename, output_path


###LLM


In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import os
# from text_to_speech import text_to_speech

class LLM:
    def __init__(self, returned_vectors) -> None:
        self.returned_vectors = returned_vectors

    def Nvidia_LLM_setup(self):
        context = "\n".join([line_with_distance[0] for line_with_distance in self.returned_vectors])

        client = ChatNVIDIA(
            model="databricks/dbrx-instruct",
            api_key = os.getenv('NVIDIA_API_KEY'),
            temperature=0.5,
            top_p=1,
            max_tokens=1024,
        )

        return context, client


    def LLM(self, question):
        context, client = self.Nvidia_LLM_setup()

        SYSTEM_PROMPT = """
                        Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
                        """
        result = []

        while question:
            USER_PROMPT = f"""
                            Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
                            You can also use the internet if the answer is not provided.
                            <context>{context}</context>
                            <question>{question}</question>
                            """
            response = []
            for chunk in client.stream([{"role": "user", "content": USER_PROMPT}]):
                print(chunk.content, end="")
                response.append(chunk.content)

            # text_to_speech(response)

            # Update context with the current response for follow-up questions
            context += " ".join(response)
            question = input("Enter your next question (or press Enter to exit): ").strip()

        print("Q&A session ended.")



###RAG


In [6]:

import torch
import os
import getpass
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch import Tensor
from torch.cuda.amp import autocast
from pymilvus import MilvusClient

  # Embedding Function "thenlper/gte-base"
    # TODO: can be changed to other model to support
    # multilingual


class RAG():
    def __init__(self, raw_text, query):
        self.raw_text = raw_text
        self.question = query
        self.client_name = None
        self.collection_name = None


    def emb_text(self, input_texts):
        """
        This function takes the raw texts from .txt file and
        using embedding model to embed.
        Using huggingface model: thenlper/gte-base

        Args:
            String input_texts: raw texts from read_docs()

        Returns:
            List Embeded texts: embedded texts
        """

        def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
            last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
            return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

        tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-base")
        model = AutoModel.from_pretrained("thenlper/gte-base")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

        batch_dict = {key: value.to("cuda") for key, value in batch_dict.items()}

        with autocast():
            outputs = model(**batch_dict)
            embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        return embeddings[0].tolist()

    def emb_text_batch(self, lines):
        with torch.no_grad():
            embeddings = self.emb_text(lines)
        return embeddings

    def milvus_setup(self, client_name, collection_name):
        self.collection_name = collection_name

        if client.has_collection(collection_name):
            client.load_collection(
                collection_name=collection_name,
        )
        else:
            client.create_collection(
                collection_name=collection_name,
                dimension=768,
                metric_type="IP",  # Inner product distance
                consistency_level="Strong",  # Strong consistency level
       	        params = {'efConstruction': 40, 'M': 1024}
       	    )

            data = []
            batch_size = 128

            text_lines = self.raw_text

            for i in tqdm(range(0, len(text_lines), batch_size), desc="Creating embeddings"):
                batch_lines = text_lines[i:i + batch_size]  # Get a batch of lines
                embeddings = self.emb_text_batch(batch_lines)  # Get embeddings for the batch
                for j, line in enumerate(batch_lines):
                    data.append({"id": i + j, "vector": embeddings, "text": line})


            client.insert(collection_name=collection_name, data=data)

        return client, collection_name

    def milvus_query(self, collection_name = None, client_name = None):
        """
        This function query the existing database
        created by previous function milvus()

        Args:
            Raw text of query question

        Returns:
            Raw text with distance
            for RAG context
        """

        """ Create new embedding"""

        milvus_client, collection_name = self.milvus_setup(client_name=client_name, collection_name=collection_name)

        search_res = client.search(
            collection_name=collection_name,
            data=[self.emb_text(self.question)],  # Use the `emb_text` function to convert the question to an embedding vector
            limit=5,  # Return top 3 results
            search_params={"metric_type": "IP", "params": {}},  # Inner product distance
            output_fields=["text"],  # Return the text field
        )

        retrieved_lines_with_distances = [(res["entity"]["text"], res["distance"]) for res in search_res[0]]

        return retrieved_lines_with_distances


###milvus_rag


In [7]:
from glob import glob
import torch.nn.functional as F
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from dotenv import load_dotenv
load_dotenv()

class txt_reader:
    def __init__(self, file_path):
        self.file_path = file_path

    def read_docs(self): # TODO: docs can be changed based on input
        with open(self.file_path, "r", encoding = 'utf-8') as file:
            lines = file.readlines()
        return [line.strip() for line in lines]

def milvus_config(filename, file_path):
    client_name = "/content/rag.db"
    collection_name = filename
    return collection_name, client_name

def mr_run(filename, file_path):
    reader = txt_reader(file_path=file_path)
    raw_text = reader.read_docs()

    collection_name, client_name = milvus_config(filename=filename, file_path=file_path);
    print(client_name)
    print(collection_name)
    question = input("Enter the question: ")
    gte = RAG(raw_text=raw_text, query=question)
    returned_vector = gte.milvus_query(collection_name=collection_name, client_name=client_name)

    llm = LLM(returned_vectors=returned_vector)
    return llm.LLM(question)


In [8]:
# !python3 /content/LLM-with-RAG/main.py

###Main


In [9]:
# import milvus_rag
# import pdf_to_txt
import os
# connections.disconnect("default")


if __name__ == "__main__":
    if os.getenv("NVIDIA_API_KEY") is None:
        NVIDIA_API_KEY = input("Add your NVIDIA key here: ")
        os.environ['NVIDIA_API_KEY'] = NVIDIA_API_KEY

    # if user want to input new pdf
    file_path = input("Enter you file path: ")
    file_extension = os.path.splitext(os.path.basename(file_path))[1]
    filename = os.path.splitext(os.path.basename(file_path))[0]
    print(file_extension)
    if file_extension == '.pdf':
        filename, txt_path = tts_run(file_path)
    else:
        filename  = filename
        txt_path = file_path

    mr_run(filename, txt_path)




Add your NVIDIA key here: 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
Enter you file path: /content/LLM-with-RAG/final.pdf
.pdf
/content/pdfReader_output/final.txt
Enter the path of your pdf file: /content/LLM-with-RAG/final.pdf
Text successfully extracted and saved to /content/pdfReader_output/final.txt
/content/rag.db
final
Enter the question: hello


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

<ipython-input-6-62d67bfd7599>:51: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Hello! How can I assist you today? Based on the context provided, it seems like you're interested in discussing social institutions, specifically gender and marriage institutions. I'm here to help answer any questions you may have about these topics or any other topic you'd like to discuss.Enter your next question (or press Enter to exit): what day is it
I'm an AI and I don't have the ability to know the current date or day. You can check a calendar or your personal device to find out the current day.

KeyboardInterrupt: Interrupted by user